# Iris Classification with PyTorch

This notebook demonstrates how to classify the Iris dataset using a simple feed-forward neural network implemented with PyTorch. TensorBoard is used for visualizing the training progress.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
import os


In [ ]:
# Load and preprocess data
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder()
y = encoder.fit_transform(y).toarray()

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
# Define the model
class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.softmax(self.fc3(x), dim=1)
        return x

model = IrisModel()


In [ ]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [ ]:
# Set up TensorBoard
writer = SummaryWriter("logs/pytorch_iris")

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        X_batch, y_batch = batch

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, torch.argmax(y_batch, dim=1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Log training loss
    writer.add_scalar('Loss/train', loss.item(), epoch)

    # Evaluate on test set
    model.eval()
    with torch.no_grad():
        test_loss = 0
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, torch.argmax(y_batch, dim=1))
            test_loss += loss.item()

        test_loss /= len(test_loader)
        writer.add_scalar('Loss/test', test_loss, epoch)

writer.close()
print("Training complete. Use TensorBoard to visualize results.")


### Visualize TensorBoard
To visualize the training logs, run the following command in your terminal:
```
tensorboard --logdir logs/pytorch_iris
```
